In [4]:
!pip install -q transformers langchain langchain-huggingface huggingface_hub

In [10]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os

In [6]:
from huggingface_hub import login

login()

In [11]:
MODEL_ID = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map = 'auto'
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [14]:
gen_pipe = pipeline(
    model = MODEL_ID,
    task = 'text-generation',
    max_new_tokens = 256,
    temperature = 0.5
)

Device set to use cpu


In [15]:
llm = HuggingFacePipeline(pipeline=gen_pipe)

In [16]:
chat_model = ChatHuggingFace(llm = llm)

In [17]:
!pip install Pydantic

In [18]:
from pydantic import Field, BaseModel

In [19]:
class Person(BaseModel):
  name:str =Field(description="name of a person")
  age:int= Field(description="Age of a person")
  city:str=Field(description="Name of the city that person belongs to")

In [21]:
parser = PydanticOutputParser(pydantic_object = Person)

In [50]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant"),
    ("human","Generate name, age, city of a frictional {place} person \n {format_instructions}")
]).partial(
    format_instructions = parser.get_format_instructions()
    )

messages = prompt.format_messages(place= 'Russia')
print(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Generate name, age, city of a frictional Russia person \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "name of a person", "title": "Name", "type": "string"}, "age": {"description": "Age of a person", "title": "Age", "type": "integer"}, "city": {"description": "Name of the city that person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}\n```', additional_kwargs={}, response_metadata={

In [51]:
chain = prompt | chat_model | parser

In [52]:
result = chain.invoke({"place":"France"})

OutputParserException: Failed to parse Person from completion {"properties": {"name": {"description": "name of a person", "title": "Name", "type": "string"}, "age": {"description": "Age of a person", "title": "Age", "type": "integer"}, "city": {"description": "Name of the city that person belongs to", "title": "City", "type": "string"}}, "required": ["name", "age", "city"]}. Got: 3 validation errors for Person
name
  Field required [type=missing, input_value={'properties': {'name': {...['name', 'age', 'city']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
age
  Field required [type=missing, input_value={'properties': {'name': {...['name', 'age', 'city']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
city
  Field required [type=missing, input_value={'properties': {'name': {...['name', 'age', 'city']}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/miss

In [31]:
print(result)

NameError: name 'result' is not defined